In [1]:
import os
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\Personal_project\\research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\Personal_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path
    transaction_dir:Path
    credit_score_dir:Path
    preprocess_dir:Path


In [5]:
from src.Banking_System.constants import *
from src.Banking_System.utils.common import read_yaml,create_directories


In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH
    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            transaction_dir=config.transaction_dir,
            credit_score_dir=config.credit_score_dir,
            preprocess_dir=config.preprocess_dir
            
            
        )
        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
import gdown
from src.Banking_System import logger
from src.Banking_System.utils.common import get_size,save_object
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
import pickle
from sklearn.model_selection import train_test_split

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    def data_preprocess(self):
        create_directories([self.config.transaction_dir])
        create_directories([self.config.credit_score_dir])
        create_directories([self.config.preprocess_dir])
        df=pd.read_csv(os.path.join(self.config.unzip_dir,'data\Credit Score Classification Dataset.csv'))
        df1=pd.read_csv(os.path.join(self.config.unzip_dir,'data\PS_20174392719_1491204439457_log.csv'))
        df_encoded=pd.get_dummies(df,columns=['Gender','Marital Status','Home Ownership'],drop_first=True)
        oe = OrdinalEncoder(categories=[['High School Diploma', "Associate's Degree", "Bachelor's Degree", "Master's Degree", 'Doctorate']])
        encoded = oe.fit_transform(df[['Education']])
        encoded_df = pd.DataFrame(encoded, columns=['Encoded_Education'])
        result_df = pd.concat([df_encoded, encoded_df], axis=1)
        df2=df1[['amount','oldbalanceOrg', 'newbalanceOrig','isFraud']]
        ss1=StandardScaler()
        ss2=StandardScaler()
        result_df[['Income','Age']]=ss1.fit_transform(result_df[['Income','Age']])
        df2[['amount','oldbalanceOrg', 'newbalanceOrig']]=ss2.fit_transform(df2[['amount','oldbalanceOrg', 'newbalanceOrig']])
        credit_train,credit_test=train_test_split(result_df,random_state=22,test_size=0.2)
        transaction_train,transaction_test=train_test_split(result_df,random_state=22,test_size=0.2)
        credit_train.to_csv(os.path.join(self.config.credit_score_dir,'credit_train.csv'))
        credit_test.to_csv(os.path.join(self.config.credit_score_dir,'credit_test.csv'))
        transaction_train.to_csv(os.path.join(self.config.transaction_dir,'transaction_train.csv'))
        transaction_test.to_csv(os.path.join(self.config.transaction_dir,'transaction_test.csv'))
        save_object(os.path.join(self.config.preprocess_dir,'creditscaler.pkl'),ss1)
        save_object(os.path.join(self.config.preprocess_dir,'transactionscaler.pkl'),ss2)
        logger.info(f"PreProcessing Completed")


        



In [9]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.data_preprocess()
except Exception as e:
    raise e

[2024-03-26 11:42:34,334: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-26 11:42:34,337: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-26 11:42:34,338: INFO: common: created directory at: artifacts]
[2024-03-26 11:42:34,339: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-26 11:42:34,340: INFO: 3840643414: Downloading data from https://drive.google.com/file/d/1VvR9L9WoUiy5vrvBYIvDlrxfq9IbYe-4/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1VvR9L9WoUiy5vrvBYIvDlrxfq9IbYe-4
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1VvR9L9WoUiy5vrvBYIvDlrxfq9IbYe-4&confirm=t&uuid=b7b571cf-9e1f-4e19-9cb2-56274ee9b946
To: c:\Users\bisht\OneDrive\Desktop\Personal_project\artifacts\data_ingestion\data.zip
100%|██████████| 1.14G/1.14G [01:54<00:00, 9.95MB/s]


[2024-03-26 11:44:31,928: INFO: 3840643414: Downloaded data from https://drive.google.com/file/d/1VvR9L9WoUiy5vrvBYIvDlrxfq9IbYe-4/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-26 11:44:45,400: INFO: common: created directory at: artifacts/data_ingestion/transaction_data]
[2024-03-26 11:44:45,402: INFO: common: created directory at: artifacts/data_ingestion/credit_score_data]
[2024-03-26 11:44:45,403: INFO: common: created directory at: artifacts/data_ingestion/preprocess]
[2024-03-26 11:44:57,718: INFO: 3840643414: PreProcessing Completed]


C:\Users\bisht\AppData\Local\Temp\ipykernel_4356\3840643414.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['amount','oldbalanceOrg', 'newbalanceOrig']]=ss2.fit_transform(df2[['amount','oldbalanceOrg', 'newbalanceOrig']])
